In [1]:
import numpy as np
import math
import time
import scipy
from math import pi, exp, factorial, sin, cos
from matplotlib import pyplot as plt

In [2]:
def QR_Householder(A): #UPGRADE: Let the algorithm accept non-square matrices?
    n = np.shape(A)[0]
    R = np.copy(A)
    Q = np.eye(n)
    V = np.zeros((n,n))
    for k in range(0,n):
        x = R[k:,k]
        e1 = np.zeros(np.size(x))
        e1[0] = 1
        v = np.sign(x[0])*np.linalg.norm(x,2)*e1 + x
        v = v/np.linalg.norm(v,2)
        V[0:(n-k),k] = v
        F = np.eye(n-k,n-k) - 2*np.outer(v,v)
        R[k:,k:] =  F@R[k:,k:]
        Q[k:,:] = F@np.copy(Q[k:,:]) #Modification to get Q.
    return R, V, np.matrix.getH(Q)
        

In [3]:
def QR_GS_gen(A): #This uses Gram-Schmitz, but can accept non-square matrix inputs.
    n = np.shape(A)
    Q = np.zeros((n[0],n[1]))
    R = np.zeros((n[1],n[1]))
    for j in range(0,n[1]):
        S = A[:,j]
        for i in range(0,j):
            R[i,j] = np.dot(np.matrix.getH(Q[:,i]),A[:,j])
            S = S - R[i,j]*Q[:,i]
        R[j,j] = np.linalg.norm(S)
        Q[:,j] = S/R[j,j]
    return Q, R

In [4]:
def QREig(A, maxiter = 1000, tol = 1e-8):
    n = np.shape(A)[0]
    eig = np.empty(n, dtype = "cfloat")
    if n == 1:
        return A[0,0]
    if n == 2:
        alpha = np.matrix.trace(A[0:2,0:2])
        beta = np.linalg.det(A[0:2,0:2])
        eig[0] = 0.5*(alpha+np.sqrt(alpha**2 - 4*beta + 0j))
        eig[1] = 0.5*(alpha-np.sqrt(alpha**2 - 4*beta + 0j))
        return eig
    if n == 3:
        for j in range(1,50):
            AO = A
            Q,R = np.linalg.qr(A)
            A = np.matmul(R,Q)
    for j in range(1,maxiter):
        AO = A
        Q,R = np.linalg.qr(A)
        A = np.matmul(R,Q)
        if np.linalg.norm(np.tril(A,-2),ord=np.inf) < tol:
            i = 0
            while i < n-1:
                if abs(A[i+1,i]) < tol**0.5:
                    eig[i] = A[i,i]
                    i += 1
                else:
                    alpha = np.matrix.trace(A[i:(i+2),i:(i+2)])
                    beta = np.linalg.det(A[i:(i+2),i:(i+2)])
                    im = np.sqrt(alpha**2 - 4*beta + 0j)
                    eig[i] = 0.5*(alpha+im)
                    eig[i+1] = 0.5*(alpha-im)
                    i += 2
            if i >= n:
                return eig
            else:
                eig[n-1] = A[n-1,n-1]
                return eig

In [5]:
B = np.random.random((6,6))
A = np.array([[2,-1,-1],[-1,2,-1],[-1,-1,2]])
print(np.linalg.eig(A)[0])
print(QREig(A))

[ 3.0000000e+00 -4.4408921e-16  3.0000000e+00]
[3.+0.j 3.+0.j 0.+0.j]
